In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xarray as xr

from mapper_functions import plot_global_contour

from my_functions import read_obsfcstana
from my_functions import read_obsfcstana_extend_datetime

In [ ]:
expt_name = 'DAv7_M36_MULTI_type_13_comb_fp_scaled'

start_date = datetime(2015, 4, 1)
end_date = datetime(2021, 4, 1)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

comp_dir = 'fp_scaled'

current_year = '2020'

In [ ]:
data_2020 = np.load(f'../test_data/{comp_dir}/{expt_name}_{start_date_str}_{end_date_str}_obsfcstana_extend_datetime_{current_year}.npz', allow_pickle=True)

In [ ]:
date_time = data_2020['date_time']
obs_species = data_2020['obs_species']
obs_tilenum = data_2020['obs_tilenum']
obs_lon = data_2020['obs_lon']
obs_lat = data_2020['obs_lat']
obs_obs = data_2020['obs_obs']
obs_fcst = data_2020['obs_fcst']
obs_ana = data_2020['obs_ana']

In [ ]:
# Calculate obs minus fcst
obs_minus_fcst = []
obs_minus_ana = []
obs_inc = []
new_date_time = []

for i in range(len(obs_obs)):
    obs_minus_fcst_chunk = obs_obs[i] - obs_fcst[i]
    obs_minus_fcst.append(obs_minus_fcst_chunk)
    obs_minus_ana_chunk = obs_obs[i] - obs_ana[i]
    obs_minus_ana.append(obs_minus_ana_chunk)
    obs_inc_chunk = obs_ana[i] - obs_fcst[i]
    obs_inc.append(obs_inc_chunk)
    
    new_date_time.append(datetime(
    date_time[i]['year'][0],
    date_time[i]['month'][0],
    date_time[i]['day'][0],
    date_time[i]['hour'][0],
    date_time[i]['min'][0],
    date_time[i]['sec'][0]
    ))

In [ ]:
# Convert the lists to numpy arrays
obs_minus_fcst = np.array(obs_minus_fcst)
obs_minus_ana = np.array(obs_minus_ana)
obs_inc = np.array(obs_inc)
new_date_time = np.array(new_date_time)

In [ ]:
# Find unique tilenum values
unique_tilenum = np.unique(obs_tilenum)

# Find the number of unique tilenum values
num_unique_tilenum = len(unique_tilenum)

# Print the number of unique tilenum values
print(f"Number of unique tilenum values: {num_unique_tilenum}")

In [ ]:
# Sort the arrays based on obs_tilenum
sort_indices = np.argsort(obs_tilenum)

sorted_date_time = new_date_time[sort_indices]
sorted_obs_tilenum = obs_tilenum[sort_indices]
sorted_obs_species = obs_species[sort_indices]
sorted_obs_obs = obs_obs[sort_indices]
sorted_obs_fcst = obs_fcst[sort_indices]
sorted_obs_ana = obs_ana[sort_indices]
sorted_obs_minus_fcst = obs_minus_fcst[sort_indices]
sorted_obs_minus_ana = obs_minus_ana[sort_indices]
sorted_obs_inc = obs_inc[sort_indices]


In [ ]:
# Find the unique tilenum values and their counts
unique_tilenum, counts = np.unique(sorted_obs_tilenum, return_counts=True)

# Calculate the indices where the groups should be split
split_indices = np.cumsum(counts)[:-1]

In [ ]:
# Split the sorted arrays based on the split indices
date_time_grouped = np.split(sorted_date_time, split_indices)
obs_species_grouped = np.split(sorted_obs_species, split_indices)
obs_obs_grouped = np.split(sorted_obs_obs, split_indices)
obs_fcst_grouped = np.split(sorted_obs_fcst, split_indices)
obs_ana_grouped = np.split(sorted_obs_ana, split_indices)
obs_minus_fcst_grouped = np.split(sorted_obs_minus_fcst, split_indices)
obs_minus_ana_grouped = np.split(sorted_obs_minus_ana, split_indices)
obs_inc_grouped = np.split(sorted_obs_inc, split_indices)

In [ ]:
# Specify the path to the netCDF file
file_path = 'DAv7_M36.inst3_1d_lndfcstana_Nt.20150901.nc4'

# Open the netCDF file
dataset = xr.open_dataset(file_path)

# Extract the lon and lat variables
lon = dataset['lon']
lat = dataset['lat']

# Assign lon and lat to each tilenum
lon_tilenum = []
lat_tilenum = []
for i in range(num_unique_tilenum):
    lon_tilenum.append(lon[int(unique_tilenum[i])])
    lat_tilenum.append(lat[int(unique_tilenum[i])])

# Convert the lists to numpy arrays
lon_tilenum = np.array(lon_tilenum)
lat_tilenum = np.array(lat_tilenum)

In [ ]:
# # Find the maximum length of sequences in the list
# max_length = max(len(item) for item in date_time_grouped)

# # Pad each sequence in the list to the maximum length
# date_time_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in date_time_grouped]
# obs_species_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_species_grouped]
# obs_obs_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_obs_grouped]
# obs_fcst_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_fcst_grouped]
# obs_ana_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_ana_grouped]
# obs_minus_fcst_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_minus_fcst_grouped]
# obs_minus_ana_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_minus_ana_grouped]
# obs_inc_grouped_padded = [np.concatenate((item, [np.nan]*(max_length - len(item)))) for item in obs_inc_grouped]


In [ ]:
# # Save the calculated values to a file
# np.savez(f'{expt_name}_{start_date_str}_{end_date_str}_obsfcstana_extend_datetime_{current_year}_grouped_by_tile.npz', 
#             date_time_grouped=date_time_grouped_padded, obs_species_grouped=obs_species_grouped_padded, obs_obs_grouped=obs_obs_grouped_padded,
#             obs_fcst_grouped=obs_fcst_grouped_padded, obs_ana_grouped=obs_ana_grouped_padded, obs_minus_fcst_grouped=obs_minus_fcst_grouped_padded,
#             obs_minus_ana_grouped=obs_minus_ana_grouped_padded, obs_inc_grouped=obs_inc_grouped_padded,
#             lat_tilenum=lat_tilenum, lon_tilenum=lon_tilenum, unique_tilenum=unique_tilenum)

In [ ]:
increments = np.load(f'../test_data/{comp_dir}/{expt_name}_{start_date_str}_{end_date_str}_increments_concat.npz', allow_pickle=True)  

# List the keys in the increments file
print(increments.files)

time_stamp = increments['time_stamp_concat']
sfmc_increment = increments['sfmc_increment_concat']   
rzmc_increment = increments['rzmc_increment_concat']

date_time_i_list = []  # Create an empty list to store date_time_i values

for ts in time_stamp:
    ts_str = ts.decode('utf-8')  # Convert bytes to string

    # Parse the string to datetime format
    date_time_i = datetime.strptime(ts_str, '%Y%m%d_%H%Mz')

    date_time_i_list.append(date_time_i)  # Append date_time_i to the list

date_time_i = np.array(date_time_i_list)  # Convert the list to a numpy array

In [ ]:
# Get core site information
import scipy.io as sio

matlab_file_path = f'../test_data/{comp_dir}/{expt_name}_CalVal_M33_SM_3h__6yr_raw_timeseries.mat'
mat_contents = sio.loadmat(matlab_file_path)

# List the keys in the mat_contents file
print(mat_contents.keys())

INSITU_id = mat_contents['INSITU_id']
id_list = INSITU_id[0].tolist()
id_list = [str(id[0]) for id in id_list]

INSITU_lat = mat_contents['INSITU_lat']
INSITU_lon = mat_contents['INSITU_lon']
ind_tile = mat_contents['ind_tile']

In [ ]:
for ii in range(len(id_list)):

    tile_number = ind_tile[0][ii] #26258
    id_number = id_list[ii]

    i = np.where(unique_tilenum == tile_number)[0][0]

    time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
    obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]
    
    new_time_stamp_s = np.array(time_stamp_s)

    # sort the arrays by time
    sort_indices_s = np.argsort(new_time_stamp_s)
    new_time_stamp_s = new_time_stamp_s[sort_indices_s]
    obs_inc_s = obs_inc_s[sort_indices_s]

    time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
    obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

    new_time_stamp_a = np.array(time_stamp_a)

    # sort the arrays by time
    sort_indices_a = np.argsort(new_time_stamp_a)
    new_time_stamp_a = new_time_stamp_a[sort_indices_a]
    obs_inc_a = obs_inc_a[sort_indices_a]

    indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
    indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
    indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

    # Extract the sfmc_increment values for the specified tilenum
    sfmc_increment_i = sfmc_increment[:, tile_number]


    bar_width = 0.15
    opacity = 0.9

    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)

    # Plot obs_inc_s in the top subplot with two y-axes
    ax1.bar(new_time_stamp_s, obs_inc_s, bar_width, alpha=opacity, color='green', label='Tb')
    ax1.set_ylabel('Tb increment (K)', weight='bold', fontsize=14)
    ax1.tick_params(axis='y', labelcolor='green')

    ax1_2 = ax1.twinx()
    ax1_2.bar(new_time_stamp_a, obs_inc_a, bar_width, alpha=opacity, color='orange', label='SFDS')
    ax1_2.set_ylabel('SFDS increment (wetness fraction)', weight='bold', fontsize=14)
    ax1_2.tick_params(axis='y', labelcolor='orange')

    # Plot the sfmc_increment values in the bottom subplot
    # ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
    ax2.bar(date_time_i[indices_s], sfmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
    ax2.bar(date_time_i[indices_a], sfmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
    ax2.bar(date_time_i[indices_sa], sfmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
    ax2.set_ylabel('Surface SM Increment ($m^3 \, m^{-3}$)', weight='bold', fontsize=14)

    ax1.yaxis.set_tick_params(labelsize=12)
    ax1_2.yaxis.set_tick_params(labelsize=12)
    ax2.yaxis.set_tick_params(labelsize=12)
    ax2.xaxis.set_tick_params(labelsize=12)

    # Add a black line at y=0 in both subplots
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax1_2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

    # Set x-axis limits
    start_date = datetime(2020, 6, 1)
    end_date = datetime(2020, 8, 31)
    ax1.set_xlim(start_date, end_date)
    ax2.set_xlim(start_date, end_date)

    # Set y-axis limits for the top subplot
    ax1.set_ylim(-15, 15)
    ax1_2.set_ylim(-0.05, 0.05)
    ax2.set_ylim(-0.06, 0.06)

    # Display the legend
    # Get the handles and labels from ax1 and ax1_2
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax1_2.get_legend_handles_labels()

    # Combine the handles and labels
    handles = handles1 + handles2
    labels = labels1 + labels2

    # Display the combined legend
    ax1.legend(handles, labels)

    # 
    ax2.legend(fontsize=18)

    # Add title to the plot
    ax1.set_title('Increment Time Series for In-Situ site ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=18)

    fig.tight_layout()
    plt.show()

    # Save the figure
    fig.savefig('increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400) #, bbox_inches='tight')

In [ ]:
ii = 6

tile_number = ind_tile[0][ii] #26258
id_number = id_list[ii]

i = np.where(unique_tilenum == tile_number)[0][0]

time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]

new_time_stamp_s = np.array(time_stamp_s)

# sort the arrays by time
sort_indices_s = np.argsort(new_time_stamp_s)
new_time_stamp_s = new_time_stamp_s[sort_indices_s]
obs_inc_s = obs_inc_s[sort_indices_s]

time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

new_time_stamp_a = np.array(time_stamp_a)

# sort the arrays by time
sort_indices_a = np.argsort(new_time_stamp_a)
new_time_stamp_a = new_time_stamp_a[sort_indices_a]
obs_inc_a = obs_inc_a[sort_indices_a]

indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

# Extract the sfmc_increment values for the specified tilenum
sfmc_increment_i = sfmc_increment[:, tile_number]


bar_width = 0.15
opacity = 0.9


In [ ]:

fig, (ax2) = plt.subplots(figsize=(20, 8))


# Plot the sfmc_increment values in the bottom subplot
# ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
ax2.bar(date_time_i[indices_s], sfmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
ax2.bar(date_time_i[indices_a], sfmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
ax2.bar(date_time_i[indices_sa], sfmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
ax2.set_ylabel('Surface SM Increment ($m^3 \, m^{-3}$)', weight='bold', fontsize=16)

ax2.yaxis.set_tick_params(labelsize=14)
ax2.xaxis.set_tick_params(labelsize=14)

# Add a black line at y=0 in both subplots
ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

# Set x-axis limits
start_date = datetime(2020, 6, 1)
end_date = datetime(2020, 8, 31)
ax2.set_xlim(start_date, end_date)

# Set y-axis limits for the top subplot
ax2.set_ylim(-0.06, 0.06)

# 
ax2.legend(fontsize=16)

# Add title to the plot
# ax2.set_title('Increment Time Series for In-Situ site ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=18)

fig.tight_layout()
plt.show()

# Save the figure
fig.savefig('sm_increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400, bbox_inches='tight')

In [ ]:
# List the keys in the mat_contents file
print(mat_contents.keys())


In [ ]:

INSITU_prcp = mat_contents['INSITU_prcp']
INSITU_sm = mat_contents['INSITU_sm']
INSITU_date_time = mat_contents['date_time_vec']

LDAS_prcp = mat_contents['LDAS_prcp_org'] 
LDAS_prcp = LDAS_prcp * 60 * 60
LDAS_sm = mat_contents['LDAS_sm_org']

print(INSITU_prcp.shape)
print(INSITU_sm.shape)
print(INSITU_date_time.shape)
print(LDAS_prcp.shape)
print(LDAS_sm.shape)

from datetime import datetime

# Parse the entire INSITU_date_time array
date_time_list = [datetime(item[0][0][0], item[1][0][0], item[2][0][0], item[3][0][0], item[4][0][0], item[5][0][0]) 
                  for sublist in INSITU_date_time for item in sublist]
# Print the first and last elements of the date_time_list
print(date_time_list[0])
print(date_time_list[-1])

# Plot surface soil moisture and precipitation for JJA 2020
start_date = datetime(2020, 6, 1)
end_date = datetime(2020, 8, 31)

# Find the indices of the date_time_list that fall within the specified date range
indices = [i for i, item in enumerate(date_time_list) if start_date <= item <= end_date]

# Extract the date_time_list values that fall within the specified date range
date_time_list_jja = [date_time_list[i] for i in indices]

site = 6

for site in range(INSITU_sm.shape[2]):
    
    # Extract the INSITU_sm values that fall within the specified date range
    INSITU_sm_jja = INSITU_sm[indices, 0 , site]

    # Extract the INSITU_prcp values that fall within the specified date range
    INSITU_prcp_jja = INSITU_prcp[indices, 0, site]

    # Extract the LDAS_sm values that fall within the specified date range
    LDAS_sm_jja = LDAS_sm[indices, 0, site]

    # Extract the LDAS_prcp values that fall within the specified date range
    LDAS_prcp_jja = LDAS_prcp[indices, 0, site]

    # Plot the surface soil moisture and precipitation values
    fig, ax1 = plt.subplots(figsize=(20, 8))

    color = 'tab:red'
    ax1.set_xlabel('time', fontsize=16)
    ax1.set_ylabel('Surface Soil Moisture ($m^3 \, m^{-3}$)', color=color, fontsize=16)
    ax1.plot(date_time_list_jja, INSITU_sm_jja, color=color)
    ax1.plot(date_time_list_jja, LDAS_sm_jja, color=color, linestyle='--')
    ax1.tick_params(axis='y', labelcolor=color)
    # ax1.set_ylim(0.05, 0.35)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Precipitation (mm)', color=color, fontsize=16)
    ax2.plot(date_time_list_jja, INSITU_prcp_jja, color=color)
    ax2.plot(date_time_list_jja, LDAS_prcp_jja, color='green', linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color)
    # ax2.set_ylim(0, 10)

    # Set x-axis limits
    start_date = datetime(2020, 5, 31)
    end_date = datetime(2020, 9, 1)
    ax2.set_xlim(start_date, end_date)

    fig.tight_layout()

In [ ]:
expt_name = 'OLv7_M36_MULTI_type_13_comb_fp_scaled'
matlab_file_path = f'../test_data/{comp_dir}/{expt_name}_CalVal_M33_SM_3h__6yr_raw_timeseries.mat'
mat_contents = sio.loadmat(matlab_file_path)

LDAS_prcp_ex1 = mat_contents['LDAS_prcp_org'] 
LDAS_prcp_ex1 = LDAS_prcp_ex1 * 60 * 60
LDAS_sm_ex1 = mat_contents['LDAS_sm_org']

expt_name = 'DAv7_M36_ASCAT_type_13_comb_fp_scaled'
matlab_file_path = f'../test_data/{comp_dir}/{expt_name}_CalVal_M33_SM_3h__6yr_raw_timeseries.mat'
mat_contents = sio.loadmat(matlab_file_path)

LDAS_sm_ex2 = mat_contents['LDAS_sm_org']

expt_name = 'DAv7_M36_SMAP_type_13_comb_fp_scaled'
matlab_file_path = f'../test_data/{comp_dir}/{expt_name}_CalVal_M33_SM_3h__6yr_raw_timeseries.mat'
mat_contents = sio.loadmat(matlab_file_path)

LDAS_sm_ex3 = mat_contents['LDAS_sm_org']

expt_name = 'DAv7_M36_MULTI_type_13_comb_fp_scaled'
matlab_file_path = f'../test_data/{comp_dir}/{expt_name}_CalVal_M33_SM_3h__6yr_raw_timeseries.mat'
mat_contents = sio.loadmat(matlab_file_path)

LDAS_sm_ex4 = mat_contents['LDAS_sm_org']

# Plot Insitu and LDAS SM and LDAS SM ex1 for JJA 2020
start_date = datetime(2020, 6, 1)
end_date = datetime(2020, 8, 31)

# Find the indices of the date_time_list that fall within the specified date range
indices = [i for i, item in enumerate(date_time_list) if start_date <= item <= end_date]

# Extract the date_time_list values that fall within the specified date range
date_time_list_jja = [date_time_list[i] for i in indices]

for site in range(INSITU_sm.shape[2]):

    # Extract the INSITU_sm values that fall within the specified date range
    INSITU_sm_jja = INSITU_sm[indices, 0 , site]

    # Extract the LDAS_sm values that fall within the specified date range
    LDAS_sm_jja = LDAS_sm[indices, 0, site]

    # Extract the LDAS_sm_ex1 values that fall within the specified date range
    LDAS_sm_ex1_jja = LDAS_sm_ex1[indices, 0, site]
    LDAS_sm_ex2_jja = LDAS_sm_ex2[indices, 0, site]
    LDAS_sm_ex3_jja = LDAS_sm_ex3[indices, 0, site]
    LDAS_sm_ex4_jja = LDAS_sm_ex4[indices, 0, site]
    INSITU_sm_jja = INSITU_sm[indices, 0, site]

    # Plot the surface soil moisture values
    fig, ax = plt.subplots(figsize=(20, 8))

    color = 'tab:red'
    ax.set_xlabel('time', fontsize=16)
    ax.set_ylabel('Surface Soil Moisture ($m^3 \, m^{-3}$)', color=color, fontsize=16)
    # ax.plot(date_time_list_jja, INSITU_sm_jja, color=color)
    ax.plot(date_time_list_jja, LDAS_sm_ex1_jja)
    ax.plot(date_time_list_jja, LDAS_sm_ex2_jja)
    ax.plot(date_time_list_jja, LDAS_sm_ex3_jja)
    ax.plot(date_time_list_jja, LDAS_sm_ex4_jja)
    ax.plot(date_time_list_jja, INSITU_sm_jja, color='gray')


    ax.tick_params(axis='y', labelcolor=color)
    ax.set_ylim(0.05, 0.35)
    ax.legend(['CNTL', 'ASC_DA', 'SMP_DA', 'MLT_DA', 'IN_SITU'])

    # Set x-axis limits
    start_date = datetime(2020, 5, 31)
    end_date = datetime(2020, 9, 1)
    ax.set_xlim(start_date, end_date)

    fig.tight_layout()


In [ ]:
ii = 3
site = ii

tile_number = ind_tile[0][ii] #26258
id_number = id_list[ii]

i = np.where(unique_tilenum == tile_number)[0][0]

time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]

new_time_stamp_s = np.array(time_stamp_s)

# sort the arrays by time
sort_indices_s = np.argsort(new_time_stamp_s)
new_time_stamp_s = new_time_stamp_s[sort_indices_s]
obs_inc_s = obs_inc_s[sort_indices_s]

time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

new_time_stamp_a = np.array(time_stamp_a)

# sort the arrays by time
sort_indices_a = np.argsort(new_time_stamp_a)
new_time_stamp_a = new_time_stamp_a[sort_indices_a]
obs_inc_a = obs_inc_a[sort_indices_a]

indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

# Extract the sfmc_increment values for the specified tilenum
sfmc_increment_i = sfmc_increment[:, tile_number]

# Extract the LDAS_sm_ex1 values that fall within the specified date range
LDAS_sm_ex1_jja = LDAS_sm_ex1[indices, 0, site]
LDAS_sm_ex2_jja = LDAS_sm_ex2[indices, 0, site]
LDAS_sm_ex3_jja = LDAS_sm_ex3[indices, 0, site]
LDAS_sm_ex4_jja = LDAS_sm_ex4[indices, 0, site]
INSITU_sm_jja = INSITU_sm[indices, 0, site]


bar_width = 0.15
opacity = 0.9

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(20, 10), sharex=True)

# Plot obs_inc_s in the top subplot with two y-axes
ax1.bar(new_time_stamp_s, obs_inc_s, bar_width, alpha=opacity, color='green', label='Tb')
ax1.set_ylabel('Tb increment (K)', fontsize=14, color='green')
ax1.tick_params(axis='y', labelcolor='green')

ax1_2 = ax1.twinx()
ax1_2.bar(new_time_stamp_a, obs_inc_a, bar_width, alpha=opacity, color='orange', label='SFDS')
ax1_2.set_ylabel('SFDS increment (wetness fraction)', fontsize=14, color='orange')
ax1_2.tick_params(axis='y', labelcolor='orange')

# Plot the sfmc_increment values in the bottom subplot
# ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
ax2.bar(date_time_i[indices_s], sfmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
ax2.bar(date_time_i[indices_a], sfmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
ax2.bar(date_time_i[indices_sa], sfmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
ax2.set_ylabel('Surface SM Increment ($m^3 \, m^{-3}$)', fontsize=14)

ax3.plot(date_time_list_jja, LDAS_sm_ex1_jja)
ax3.plot(date_time_list_jja, LDAS_sm_ex4_jja)
ax3.plot(date_time_list_jja, INSITU_sm_jja, color='gray')
ax3.set_ylabel('Surface Soil Moisture ($m^3 \, m^{-3}$)', fontsize=14)
ax3.legend(['CNTL', 'MLT_DA', 'IN_SITU'])

ax1.yaxis.set_tick_params(labelsize=12)
ax1_2.yaxis.set_tick_params(labelsize=12)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=12)
ax3.yaxis.set_tick_params(labelsize=12)
ax3.xaxis.set_tick_params(labelsize=14)

# Add a black line at y=0 in both subplots
ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
ax1_2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

# Set x-axis limits
start_date = datetime(2020, 5, 31)
end_date = datetime(2020, 9, 1)
ax1.set_xlim(start_date, end_date)
ax2.set_xlim(start_date, end_date)

# Set y-axis limits for the top subplot
ax1.set_ylim(-15, 15)
ax1_2.set_ylim(-0.05, 0.05)
ax2.set_ylim(-0.06, 0.06)

# Display the legend
# Get the handles and labels from ax1 and ax1_2
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax1_2.get_legend_handles_labels()

# Combine the handles and labels
handles = handles1 + handles2
labels = labels1 + labels2

# Display the combined legend
ax1.legend(handles, labels)

# 
ax2.legend()

# Add title to the plot
ax1.set_title('Increment Time Series for In-Situ site ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=18)

fig.tight_layout()
plt.show()

# Save the figure
# fig.savefig('increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400) #, bbox_inches='tight')

In [ ]:
for ii in [2, 3, 6, 7, 8, 10]:
    site = ii

    tile_number = ind_tile[0][ii] #26258
    id_number = id_list[ii]

    i = np.where(unique_tilenum == tile_number)[0][0]

    time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
    obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]

    new_time_stamp_s = np.array(time_stamp_s)

    # sort the arrays by time
    sort_indices_s = np.argsort(new_time_stamp_s)
    new_time_stamp_s = new_time_stamp_s[sort_indices_s]
    obs_inc_s = obs_inc_s[sort_indices_s]

    time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
    obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

    new_time_stamp_a = np.array(time_stamp_a)

    # sort the arrays by time
    sort_indices_a = np.argsort(new_time_stamp_a)
    new_time_stamp_a = new_time_stamp_a[sort_indices_a]
    obs_inc_a = obs_inc_a[sort_indices_a]

    indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
    indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
    indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

    # Extract the sfmc_increment values for the specified tilenum
    sfmc_increment_i = sfmc_increment[:, tile_number]

    # Extract the LDAS_sm_ex1 values that fall within the specified date range
    LDAS_sm_ex1_jja = LDAS_sm_ex1[indices, 0, site]
    LDAS_sm_ex2_jja = LDAS_sm_ex2[indices, 0, site]
    LDAS_sm_ex3_jja = LDAS_sm_ex3[indices, 0, site]
    LDAS_sm_ex4_jja = LDAS_sm_ex4[indices, 0, site]
    INSITU_sm_jja = INSITU_sm[indices, 0, site]


    bar_width = 0.15
    opacity = 0.9

    fig, (ax2, ax3) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)

    # Plot obs_inc_s in the top subplot with two y-axes
    ax1.bar(new_time_stamp_s, obs_inc_s, bar_width, alpha=opacity, color='green', label='Tb')
    ax1.set_ylabel('Tb increment (K)', fontsize=14, color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    ax1_2 = ax1.twinx()
    ax1_2.bar(new_time_stamp_a, obs_inc_a, bar_width, alpha=opacity, color='orange', label='SFDS')
    ax1_2.set_ylabel('SFDS increment (wetness fraction)', fontsize=14, color='orange')
    ax1_2.tick_params(axis='y', labelcolor='orange')

    # Plot the sfmc_increment values in the bottom subplot
    # ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
    ax2.bar(date_time_i[indices_s], sfmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
    ax2.bar(date_time_i[indices_a], sfmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
    ax2.bar(date_time_i[indices_sa], sfmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
    ax2.set_ylabel('Surface SM Increment ($m^3 \, m^{-3}$)', fontsize=14)


    ax3.plot(date_time_list_jja, INSITU_sm_jja, color='gray')
    ax3.plot(date_time_list_jja, LDAS_sm_ex1_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex2_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex3_jja)
    #ax3.plot(date_time_list_jja, LDAS_sm_ex4_jja)
    
    ax3.set_ylabel('Surface Soil Moisture ($m^3 \, m^{-3}$)', fontsize=14)
    ax3.legend(['IN_SITU', 'CNTL', 'ASC_DA', 'SMP_DA', 'MLT_DA'])

    ax1.yaxis.set_tick_params(labelsize=12)
    ax1_2.yaxis.set_tick_params(labelsize=12)
    ax2.yaxis.set_tick_params(labelsize=12)
    ax2.xaxis.set_tick_params(labelsize=12)
    ax3.yaxis.set_tick_params(labelsize=12)
    ax3.xaxis.set_tick_params(labelsize=14)

    # Add a black line at y=0 in both subplots
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax1_2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

    # Set x-axis limits
    start_date = datetime(2020, 5, 31)
    end_date = datetime(2020, 9, 1)
    ax1.set_xlim(start_date, end_date)
    ax2.set_xlim(start_date, end_date)

    # Set y-axis limits for the top subplot
    ax1.set_ylim(-15, 15)
    ax1_2.set_ylim(-0.05, 0.05)
    ax2.set_ylim(-0.06, 0.06)

    # Display the legend
    # Get the handles and labels from ax1 and ax1_2
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax1_2.get_legend_handles_labels()

    # Combine the handles and labels
    handles = handles1 + handles2
    labels = labels1 + labels2

    # Display the combined legend
    ax1.legend(handles, labels)

    # 
    ax2.legend()

    # Add title to the plot
    ax2.set_title('Increment Time Series for In-Situ site ' + str(ii) + ' ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=14)

    fig.tight_layout()
    plt.show()

    # Save the figure
    # fig.savefig('increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400) #, bbox_inches='tight')

In [ ]:
for ii in [2, 3, 6, 7, 8, 10]:
    site = ii

    tile_number = ind_tile[0][ii] #26258
    id_number = id_list[ii]

    i = np.where(unique_tilenum == tile_number)[0][0]

    time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
    obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]

    new_time_stamp_s = np.array(time_stamp_s)

    # sort the arrays by time
    sort_indices_s = np.argsort(new_time_stamp_s)
    new_time_stamp_s = new_time_stamp_s[sort_indices_s]
    obs_inc_s = obs_inc_s[sort_indices_s]

    time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
    obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

    new_time_stamp_a = np.array(time_stamp_a)

    # sort the arrays by time
    sort_indices_a = np.argsort(new_time_stamp_a)
    new_time_stamp_a = new_time_stamp_a[sort_indices_a]
    obs_inc_a = obs_inc_a[sort_indices_a]

    indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
    indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
    indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

    # Extract the sfmc_increment values for the specified tilenum
    sfmc_increment_i = sfmc_increment[:, tile_number]

    # Extract the LDAS_sm_ex1 values that fall within the specified date range
    LDAS_sm_ex1_jja = LDAS_sm_ex1[indices, 0, site]
    LDAS_sm_ex2_jja = LDAS_sm_ex2[indices, 0, site]
    LDAS_sm_ex3_jja = LDAS_sm_ex3[indices, 0, site]
    LDAS_sm_ex4_jja = LDAS_sm_ex4[indices, 0, site]
    INSITU_sm_jja = INSITU_sm[indices, 0, site]


    bar_width = 0.15
    opacity = 0.9

    fig, (ax2, ax3) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)

    # Plot obs_inc_s in the top subplot with two y-axes
    ax1.bar(new_time_stamp_s, obs_inc_s, bar_width, alpha=opacity, color='green', label='Tb')
    ax1.set_ylabel('Tb increment (K)', fontsize=14, color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    ax1_2 = ax1.twinx()
    ax1_2.bar(new_time_stamp_a, obs_inc_a, bar_width, alpha=opacity, color='orange', label='SFDS')
    ax1_2.set_ylabel('SFDS increment (wetness fraction)', fontsize=14, color='orange')
    ax1_2.tick_params(axis='y', labelcolor='orange')

    # Plot the sfmc_increment values in the bottom subplot
    # ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
    ax2.bar(date_time_i[indices_s], sfmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
    ax2.bar(date_time_i[indices_a], sfmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
    ax2.bar(date_time_i[indices_sa], sfmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
    ax2.set_ylabel('Surface SM Increment ($m^3 \, m^{-3}$)', fontsize=14)


    ax3.plot(date_time_list_jja, INSITU_sm_jja, color='gray')
    ax3.plot(date_time_list_jja, LDAS_sm_ex1_jja)
    # ax3.plot(date_time_list_jja, LDAS_sm_ex2_jja)
    # ax3.plot(date_time_list_jja, LDAS_sm_ex3_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex4_jja)
    
    ax3.set_ylabel('Surface Soil Moisture ($m^3 \, m^{-3}$)', fontsize=14)
    ax3.legend(['IN_SITU', 'CNTL', 'MLT_DA'])

    ax1.yaxis.set_tick_params(labelsize=12)
    ax1_2.yaxis.set_tick_params(labelsize=12)
    ax2.yaxis.set_tick_params(labelsize=12)
    ax2.xaxis.set_tick_params(labelsize=12)
    ax3.yaxis.set_tick_params(labelsize=12)
    ax3.xaxis.set_tick_params(labelsize=14)

    # Add a black line at y=0 in both subplots
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax1_2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

    # Set x-axis limits
    start_date = datetime(2020, 5, 31)
    end_date = datetime(2020, 9, 1)
    ax1.set_xlim(start_date, end_date)
    ax2.set_xlim(start_date, end_date)

    # Set y-axis limits for the top subplot
    ax1.set_ylim(-15, 15)
    ax1_2.set_ylim(-0.05, 0.05)
    ax2.set_ylim(-0.06, 0.06)

    # Display the legend
    # Get the handles and labels from ax1 and ax1_2
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax1_2.get_legend_handles_labels()

    # Combine the handles and labels
    handles = handles1 + handles2
    labels = labels1 + labels2

    # Display the combined legend
    ax1.legend(handles, labels)

    # 
    ax2.legend()

    # Add title to the plot
    ax2.set_title('Increment Time Series for In-Situ site ' + str(ii) + ' ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=14)

    fig.tight_layout()
    plt.show()

    # Save the figure
    # fig.savefig('increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400) #, bbox_inches='tight')

In [ ]:
for ii in [6, 7, 8]:
    site = ii

    tile_number = ind_tile[0][ii]
    id_number = id_list[ii]

    i = np.where(unique_tilenum == tile_number)[0][0]

    time_stamp_s = date_time_grouped[i][obs_species_grouped[i] < 5]
    obs_inc_s = obs_inc_grouped[i][obs_species_grouped[i] < 5]

    new_time_stamp_s = np.array(time_stamp_s)

    # sort the arrays by time
    sort_indices_s = np.argsort(new_time_stamp_s)
    new_time_stamp_s = new_time_stamp_s[sort_indices_s]
    obs_inc_s = obs_inc_s[sort_indices_s]

    time_stamp_a = date_time_grouped[i][obs_species_grouped[i] > 4]
    obs_inc_a = obs_inc_grouped[i][obs_species_grouped[i] > 4]

    new_time_stamp_a = np.array(time_stamp_a)

    # sort the arrays by time
    sort_indices_a = np.argsort(new_time_stamp_a)
    new_time_stamp_a = new_time_stamp_a[sort_indices_a]
    obs_inc_a = obs_inc_a[sort_indices_a]

    indices_s = np.where(np.isin(date_time_i, new_time_stamp_s))[0].astype(int)
    indices_a = np.where(np.isin(date_time_i, new_time_stamp_a))[0].astype(int)
    indices_sa = np.intersect1d(np.where(np.isin(date_time_i, new_time_stamp_s))[0], np.where(np.isin(date_time_i, new_time_stamp_a))[0]).astype(int)

    # Extract the sfmc_increment values for the specified tilenum
    rzmc_increment_i = rzmc_increment[:, tile_number]

    # Extract the LDAS_sm_ex1 values that fall within the specified date range
    LDAS_sm_ex1_jja = LDAS_sm_ex1[indices, 1, site]
    LDAS_sm_ex2_jja = LDAS_sm_ex2[indices, 1, site]
    LDAS_sm_ex3_jja = LDAS_sm_ex3[indices, 1, site]
    LDAS_sm_ex4_jja = LDAS_sm_ex4[indices, 1, site]
    INSITU_sm_jja = INSITU_sm[indices, 1, site]


    bar_width = 0.15
    opacity = 0.9

    fig, (ax2, ax3) = plt.subplots(nrows=2, ncols=1, figsize=(20, 10), sharex=True)

    # Plot obs_inc_s in the top subplot with two y-axes
    ax1.bar(new_time_stamp_s, obs_inc_s, bar_width, alpha=opacity, color='green', label='Tb')
    ax1.set_ylabel('Tb increment (K)', fontsize=14, color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    ax1_2 = ax1.twinx()
    ax1_2.bar(new_time_stamp_a, obs_inc_a, bar_width, alpha=opacity, color='orange', label='SFDS')
    ax1_2.set_ylabel('SFDS increment (wetness fraction)', fontsize=14, color='orange')
    ax1_2.tick_params(axis='y', labelcolor='orange')

    # Plot the sfmc_increment values in the bottom subplot
    # ax2.bar(date_time_i, sfmc_increment_i, bar_width, color='blue', alpha=0.2, label='all_sfmc_inc')
    ax2.bar(date_time_i[indices_s], rzmc_increment_i[indices_s], bar_width, color='green', label='SMP caused')
    ax2.bar(date_time_i[indices_a], rzmc_increment_i[indices_a], bar_width,  color='orange', label='ASC caused')
    ax2.bar(date_time_i[indices_sa], rzmc_increment_i[indices_sa], bar_width, color='red', label='Both caused')
    ax2.set_ylabel('Rootzone SM Increment ($m^3 \, m^{-3}$)', fontsize=14)


    ax3.plot(date_time_list_jja, INSITU_sm_jja, color='gray')
    ax3.plot(date_time_list_jja, LDAS_sm_ex1_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex2_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex3_jja)
    ax3.plot(date_time_list_jja, LDAS_sm_ex4_jja)
    
    ax3.set_ylabel('Rootzone Soil Moisture ($m^3 \, m^{-3}$)', fontsize=14)
    ax3.legend(['IN_SITU', 'CNTL', 'ASC_DA', 'SMP_DA', 'MLT_DA'])

    ax1.yaxis.set_tick_params(labelsize=12)
    ax1_2.yaxis.set_tick_params(labelsize=12)
    ax2.yaxis.set_tick_params(labelsize=12)
    ax2.xaxis.set_tick_params(labelsize=12)
    ax3.yaxis.set_tick_params(labelsize=12)
    ax3.xaxis.set_tick_params(labelsize=14)

    # Add a black line at y=0 in both subplots
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax1_2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

    # Set x-axis limits
    start_date = datetime(2020, 5, 31)
    end_date = datetime(2020, 9, 1)
    ax1.set_xlim(start_date, end_date)
    ax2.set_xlim(start_date, end_date)

    # Set y-axis limits for the top subplot
    # ax1.set_ylim(-15, 15)
    ax2.set_ylim(-0.005, 0.005)

    # Display the legend
    # Get the handles and labels from ax1 and ax1_2
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax1_2.get_legend_handles_labels()

    # Combine the handles and labels
    handles = handles1 + handles2
    labels = labels1 + labels2

    # Display the combined legend
    ax1.legend(handles, labels)

    # 
    ax2.legend()

    # Add title to the plot
    ax2.set_title('Increment Time Series for In-Situ site ' + str(ii) + ' ' + str(id_number)+ ', JJA 2020', weight='bold', fontsize=14)

    fig.tight_layout()
    plt.show()

    # Save the figure
    # fig.savefig('increment_time_series_INSITU_id_'+str(id_number)+'_JJA_2020.png', dpi=400) #, bbox_inches='tight')